# Building the design matrix for TBSS

Modeling the interaction between ApoE and centiloids on DWI

In [52]:
%run /home/grg/git/alfa/nilearn-helper.py
%run /home/grg/git/alfa/build_matrix.py
%matplotlib inline

In [17]:
import json, string
import os.path as op
from glob import glob
dd = '/home/grg/SPM/2018'
data_dir = '/home/grg/TBSS/AMYLOID/amylstaging/origdata/'
subjects = sorted([string.atoi(op.basename(e).split('_')[0]) for e in glob(op.join(data_dir, '*.nii.gz'))]) #.difference(set(failed)))
len(subjects)
print subjects

[10019, 10044, 10065, 10070, 10137, 10162, 10200, 10225, 10235, 10245, 10319, 10329, 10424, 10515, 10560, 10613, 10627, 10649, 10696, 10711, 10724, 10779, 10810, 10881, 10949, 11042, 11055, 11111, 11114, 11125, 11127, 11132, 11248, 11257, 11262, 11273, 11291, 11387, 11397, 11478, 11514, 11540, 11587, 11593, 11656, 11711, 11742, 11745, 11747, 11829, 11866, 12118, 12184, 12269, 12279, 12304, 12308, 12627, 12741, 12769, 12778, 12783, 12810, 12991, 13035, 13041, 13059, 13105, 13107, 13166, 13193, 13244, 13259, 21136, 21138, 21156, 44008, 44158, 44214, 44231, 44301, 44304, 44329, 44349, 44363, 44396, 44660, 55038, 55210, 55607, 55630, 55777, 55797, 55834, 55854, 66008, 66027, 66045, 66067, 66100, 66162, 66224, 66237, 66249, 66296, 66301, 66488, 66489, 77012, 77101, 77104, 77164, 77224, 77341]


In [83]:
variables = ['subject', 'age', 'centiloids', 'apoe']
cov = pd.DataFrame(pd.read_excel(osp.join(dd, 'covariates.xlsx'), converters={'subject':int, 'apoe':int}), columns=variables).set_index('subject')
cov = cov.loc[subjects]

In [67]:
cov = pd.DataFrame(pd.read_excel(osp.join(dd, 'covariates.xlsx'), converters={'subject':int, 'apoe':int})) #.set_index('subject')
print subjects
images = [e for e in sorted(glob(osp.join(data_dir, '*.nii.gz'))) if string.atoi(e.split('/')[-1].split('_')[0]) in subjects]
print len(images)

data = build_matrix(images, ['age', 'gender', 'apoe', 'centiloids'], cov, subjects=subjects)
data = build_dummy(data, 'apoe')
data = build_interaction(data, 'centiloids', 'apoe')
del data['apoe']
#del data['stage']

data.head()

[10019, 10044, 10065, 10070, 10137, 10162, 10200, 10225, 10235, 10245, 10319, 10329, 10424, 10515, 10560, 10613, 10627, 10649, 10696, 10711, 10724, 10779, 10810, 10881, 10949, 11042, 11055, 11111, 11114, 11125, 11127, 11132, 11248, 11257, 11262, 11273, 11291, 11387, 11397, 11478, 11514, 11540, 11587, 11593, 11656, 11711, 11742, 11745, 11747, 11829, 11866, 12118, 12184, 12269, 12279, 12304, 12308, 12627, 12741, 12769, 12778, 12783, 12810, 12991, 13035, 13041, 13059, 13105, 13107, 13166, 13193, 13244, 13259, 21136, 21138, 21156, 44008, 44158, 44214, 44231, 44301, 44304, 44329, 44349, 44363, 44396, 44660, 55038, 55210, 55607, 55630, 55777, 55797, 55834, 55854, 66008, 66027, 66045, 66067, 66100, 66162, 66224, 66237, 66249, 66296, 66301, 66488, 66489, 77012, 77101, 77104, 77164, 77224, 77341]
114


,images,age,gender,centiloids,0,1,2,centiloids0,centiloids1,centiloids2
0,/home/grg/TBSS/AMYLOID/amylstaging/origdata/10...,63.422313,2,-7.62,1,0,0,-7.62,-0.0,-0.00
1,/home/grg/TBSS/AMYLOID/amylstaging/origdata/10...,54.757016,1,-1.84,1,0,0,-1.84,-0.0,-0.00
2,/home/grg/TBSS/AMYLOID/amylstaging/origdata/10...,50.390144,2,-0.16,0,0,1,-0.00,-0.0,-0.16
3,/home/grg/TBSS/AMYLOID/amylstaging/origdata/10...,54.151951,1,-4.12,1,0,0,-4.12,-0.0,-0.00
4,/home/grg/TBSS/AMYLOID/amylstaging/origdata/10...,52.878850,1,-17.65,1,0,0,-17.65,-0.0,-0.00


In [85]:
data = pd.DataFrame(pd.read_excel(osp.join(dd, 'covariates.xlsx'), converters={'subject':int, 'apoe':int})) #.set_index('subject')
del data['stage']
del data['subject']
#data = build_dummy(data, 'apoe')
data = build_interaction(data, 'centiloids', 'apoe')
del data['apoe']
mat = build_tbss_matrix(data)
con = tbss_covariates_simple_contrasts(list(data.columns))
pair_con = tbss_2vs2_contrasts(['centiloids0', 'centiloids1', 'centiloids2'], list(data.columns))
con.extend(pair_con)
con

[(u'age(+)', [1, 0, 0, 0, 0, 0]),
 (u'age(-)', [-1, 0, 0, 0, 0, 0]),
 (u'gender(+)', [0, 1, 0, 0, 0, 0]),
 (u'gender(-)', [0, -1, 0, 0, 0, 0]),
 (u'centiloids(+)', [0, 0, 1, 0, 0, 0]),
 (u'centiloids(-)', [0, 0, -1, 0, 0, 0]),
 ('centiloids0(+)', [0, 0, 0, 1, 0, 0]),
 ('centiloids0(-)', [0, 0, 0, -1, 0, 0]),
 ('centiloids1(+)', [0, 0, 0, 0, 1, 0]),
 ('centiloids1(-)', [0, 0, 0, 0, -1, 0]),
 ('centiloids2(+)', [0, 0, 0, 0, 0, 1]),
 ('centiloids2(-)', [0, 0, 0, 0, 0, -1]),
 ('centiloids0>centiloids1', [0, 0, 0, 1, -1, 0]),
 ('centiloids0>centiloids2', [0, 0, 0, 1, 0, -1]),
 ('centiloids1>centiloids0', [0, 0, 0, -1, 1, 0]),
 ('centiloids1>centiloids2', [0, 0, 0, 0, 1, -1]),
 ('centiloids2>centiloids0', [0, 0, 0, -1, 0, 1]),
 ('centiloids2>centiloids1', [0, 0, 0, 0, -1, 1])]

In [86]:
mat = build_tbss_matrix(data)
contrasts = tbss_covariates_simple_contrasts(data.columns)
pair_contrasts = tbss_2vs2_contrasts(['centiloids%i'%i for i in range(0,3)], list(data.columns))
contrasts.extend(pair_contrasts)
print build_tbss_contrasts(contrasts)
#dump(contrasts, ['/tmp/con%s.con'%i for i in xrange(1)])

# writing files
#w = open('/tmp/amylstaging_interaction_apoe_centiloids.mat','w')
#w.write(mat)
#w.close()
#mat

/NumWaves 6
/ContrastName1 age(+)
/ContrastName2 age(-)
/ContrastName3 gender(+)
/ContrastName4 gender(-)
/ContrastName5 centiloids(+)
/ContrastName6 centiloids(-)
/ContrastName7 centiloids0(+)
/ContrastName8 centiloids0(-)
/ContrastName9 centiloids1(+)
/ContrastName10 centiloids1(-)
/ContrastName11 centiloids2(+)
/ContrastName12 centiloids2(-)
/ContrastName13 centiloids0>centiloids1
/ContrastName14 centiloids0>centiloids2
/ContrastName15 centiloids1>centiloids0
/ContrastName16 centiloids1>centiloids2
/ContrastName17 centiloids2>centiloids0
/ContrastName18 centiloids2>centiloids1
/NumContrasts 18
/Matrix
1 0 0 0 0 0
-1 0 0 0 0 0
0 1 0 0 0 0
0 -1 0 0 0 0
0 0 1 0 0 0
0 0 -1 0 0 0
0 0 0 1 0 0
0 0 0 -1 0 0
0 0 0 0 1 0
0 0 0 0 -1 0
0 0 0 0 0 1
0 0 0 0 0 -1
0 0 0 1 -1 0
0 0 0 1 0 -1
0 0 0 -1 1 0
0 0 0 0 1 -1
0 0 0 -1 0 1
0 0 0 0 -1 1


In [35]:
commands = []
def dump_commands(commands, fp):
    def chunks(l, n): 
        for i in range(0, len(l), n): 
            yield l[i:i + n]
    for each, f in zip(chunks(commands, len(commands)/len(fp)), fp):
        print each, f
        w = open(f, 'w')
        for e in each:
            w.write('%s\n'%e)
        w.close()

for t in ['MD', 'FA']:
    for i in xrange(1):        
        cmd = 'mailx -s "%s - %s - con%s - tbss_%s_ApoeCentilInteraction_con%s" < /dev/null "goperto@barcelonabeta.org"'%(t, contrasts[i][0], i, t, i)
        #commands.append(cmd)
        print cmd
        cmd = 'randomise -i all_%s_skeletonised -o tbss_%s_ApoeCentilInteraction -m mean_FA_skeleton_mask -d interaction/amylstaging_interaction_apoe_centiloids.mat -t interaction/%s -n 1000 --T2 -V'\
            %(t, t, 'con%s.con'%i)
        commands.append(cmd)

dump_commands(commands, ['/tmp/script%s.sh'%i for i in xrange(1)])

mailx -s "MD - age(+) - con0 - tbss_MD_ApoeCentilInteraction_con0" < /dev/null "goperto@barcelonabeta.org"
mailx -s "FA - age(+) - con0 - tbss_FA_ApoeCentilInteraction_con0" < /dev/null "goperto@barcelonabeta.org"
['randomise -i all_MD_skeletonised -o tbss_MD_ApoeCentilInteraction -m mean_FA_skeleton_mask -d interaction/amylstaging_interaction_apoe_centiloids.mat -t interaction/con0.con -n 1000 --T2 -V', 'randomise -i all_FA_skeletonised -o tbss_FA_ApoeCentilInteraction -m mean_FA_skeleton_mask -d interaction/amylstaging_interaction_apoe_centiloids.mat -t interaction/con0.con -n 1000 --T2 -V'] /tmp/script0.sh
